# Introduction to local models

In this tutorial, we will investigate the possibilities of fitting data to local models and compare their performances to a single global model.

In [ ]:
import pymks
import matplotlib.pyplot as plt
import matplotlib as mpl

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

Similarly to previous examples, we will generate microstructures and evaluate responses of interest. In this case, we will be generating three classes of two phase microstructures (equiaxed, horizontal fibers and vertical fibers). Each class will produce 200 microstructures, for a total of 600 microstructures.

In [ ]:
n_samples = [200, 200, 200]
grain_size = [(14, 14), (4, 49), (47, 6)]

from pymks.datasets import make_elastic_stress_random

X, y = make_elastic_stress_random(n_samples=n_samples, size=(51, 51),
                                            grain_size=grain_size, seed=100)

Let's get a quick visualization of the structures we're generating.

In [ ]:
from pymks.tools import draw_microstructures

draw_microstructures(X[[100,300,500]])

Now we need to calculate the two point statistics of the structures using a primitive basis. We also assume that both axes are periodic.

In [ ]:
from pymks import PrimitiveBasis
from pymks.stats import correlate

p_basis = PrimitiveBasis(n_states=2, domain=[0, 1])
X_corr = correlate(X, p_basis, periodic_axes=[0, 1])

print(X_corr.shape)

The dimensions of our correlations are 600 x 51 x 51 x 3. 600 is the number of microstructures in our dataset, 51 x 51 are the dimensions of each microstructure, and 3 is the number of correlations we capture (Phase 1 autocorrelations, Phase 2 autocorrelations, and Phase 1 - Phase 2 cross-correlations). However, we only need one of these correlations to fully define the correlations of two phase structures, so we'll select the Phase 1 autocorrelations as our inputs.

In [ ]:
autos=X_corr[:,:,:,0].reshape(X_corr.shape[0],X_corr.shape[1]*X_corr.shape[2])

print(autos.shape)

Now the dimensions are 600 x 51^2, which is what we want. Next, let's divide the dataset into a training set and a testing set. For this exercise, the train/test split will be 400/200 and will be performed randomly.

In [ ]:
import numpy as np

n_train=400

total_inds=np.arange(autos.shape[0])

train_inds=np.sort(np.random.choice(total_inds,n_train,replace=False))
test_inds=np.delete(total_inds,train_inds)

Now that we've decided which indices belong to the training set and which belong to the testing set, we can assign inputs and responses to the sets. Responses do not need to be modified once they have been assigned, but the inputs need to be transformed into a lower dimensional representation; otherwise, the model will be too bulky.

In [ ]:
autos_train=autos[train_inds]
autos_test=autos[test_inds]
y_train=y[train_inds]
y_test=y[test_inds]

To reduce the dimensionality of the input space, we perform PCA.

In [ ]:
tps_mean=autos_train.mean(axis=0)

autos_train_meanshift=autos_train-tps_mean
autos_test_meanshift=autos_test-tps_mean

u,s,v=np.linalg.svd(autos_train_meanshift,full_matrices=False)

n_components=10

scores=u*s
X_train=scores[:,0:n_components]
basis=v[0:n_components,:].T

X_test=np.linalg.lstsq(a=basis,b=autos_test_meanshift.T)[0].T

Now that inputs and responses have been fully defined for both the training set and the testing set, let's build model. Here we will use a Gaussian process regression (GPR) model. In GPR, a prior over functions is combined with data to obtain a posterior over functions, which are normally distributed for all collections of input variables: 

In [ ]:
from plot_gpr_prior_posterior import plot_gpr

plot_gpr()

The type of covariance (or kernel) function selected for the GPR model has a large effect on its behavior, since it defines how "similar" different data points are. We can import different kernel function types and also construct a custom kernel function.

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import WhiteKernel

kernel=1.0 * Matern(length_scale=1*np.ones(10), length_scale_bounds=(1e2, 1e8),
                            nu=0.5)+RBF(length_scale=1*np.ones(10),length_scale_bounds=(1e2,1e8))+WhiteKernel()

Before we build our model, let's visualize our input space.

In [ ]:
plt.figure()
ax = plt.subplot(111)
ax.plot(X_train[:,0],X_train[:,1],'b.')
plt.xlabel('PC1')
plt.ylabel('PC2')
ax.grid(False)
plt.title('Training set inputs in the PC1-PC2 space')

There appear to be three distinct clusters in the input space, but for now we'll build a single model over all the inputs. Next, we'll build a model over the training set and predict the responses for the test set. We'll measure the percent mean absolute error as well as the time necessary to fit the GPR model to the training data.

In [ ]:
import time

gp=GaussianProcessRegressor(kernel=kernel)
time_start=time.clock()
gp=gp.fit(X_train,y_train)
time_stop=time.clock()
elapsed_time=np.round(time_stop-time_start,2)
preds=gp.predict(X_test)
pmae=np.round(100*np.sum(abs(y_test-preds))/np.sum(y_test),3)

plt.figure()
ax = plt.subplot(111)
ax.plot(y_test,y_test,'k')
ax.plot(y_test,preds,'b.')
plt.xlabel('FE simulation value')
plt.ylabel('GPR model prediction')
ax.grid(False)
plt.title('Macroscopic stress of the test set microstructures (200)')

print('The model took {} seconds to fit and produced a percent mean absolute error of {}%'.format(elapsed_time,pmae))

Not a bad result, but let's see if we can improve it. Here we introduce the concept of local models: rather than building a single model over the entire input space, we build multiple models that cover different parts of the input space. K-means clustering attempts to minimize the total in-group variance given a number of clusters, and is thus an ideal algorithm for our purposes. Since there appeared to be three clusters in the input space, let's specify the number of clusters as three.

In [ ]:
from sklearn.cluster import KMeans

n_clusters=3

X_train_local={}
X_test_local={}
y_train_local={}
y_test_local={}
preds={}
err={}
elapsed_time=np.zeros(n_clusters)

cluster_obj=KMeans(n_clusters=n_clusters).fit(X_train)
clusters_train=cluster_obj.predict(X_train)
clusters_test=cluster_obj.predict(X_test)

for i in range(n_clusters):
    X_train_local[i]=X_train[np.where(clusters_train==i)[0]]
    X_test_local[i]=X_test[np.where(clusters_test==i)[0]]
    y_train_local[i]=y_train[np.where(clusters_train==i)[0]]
    y_test_local[i]=y_test[np.where(clusters_test==i)[0]]
    
plt.figure()
ax = plt.subplot(111)
for i in range(n_clusters):
    ax.plot(X_train_local[i][:,0],X_train_local[i][:,1],'.')
    
plt.xlabel('PC1')
plt.ylabel('PC2')
ax.grid(False)
plt.title('Training set inputs in the PC1-PC2 space')

plt.figure()
ax = plt.subplot(111)
for i in range(n_clusters):
    ax.plot(X_test_local[i][:,0],X_test_local[i][:,1],'.')
    
plt.xlabel('PC1')
plt.ylabel('PC2')
ax.grid(False)
plt.title('Testing set inputs in the PC1-PC2 space')

We get three clusters, but it doesn't look quite right. Nevertheless, let's keep it this way for now. We'll fit our three models and predict over the test set.

In [ ]:
for i in range(n_clusters):
    gp=GaussianProcessRegressor(kernel=kernel)
    time_start=time.clock()
    gp=gp.fit(X_train_local[i],y_train_local[i])
    time_stop=time.clock()
    elapsed_time[i]=time_stop-time_start
    preds[i]=gp.predict(X_test_local[i])
    err[i]=abs(y_test_local[i]-preds[i])
    
for i in range(n_clusters):
    if i==0:
        error_total=err[i]
        y_test_total=y_test_local[i]
        preds_total=preds[i]
    else:
        error_total=np.append(error_total,err[i])
        y_test_total=np.append(y_test_total,y_test_local[i])
        preds_total=np.append(preds_total,preds[i])

elapsed_time_sum=np.round(np.sum(elapsed_time),2)
pmae=np.round(100*np.sum(error_total)/np.sum(y_test_total),3)

plt.figure()
ax = plt.subplot(111)
ax.plot(y_test_total,y_test_total,'k')
ax.plot(y_test_total,preds_total,'b.')
plt.xlabel('FE simulation value')
plt.ylabel('GPR model prediction')
ax.grid(False)
plt.title('Macroscopic stress of the test set microstructures (200)')

print('The model took {} seconds to fit and produced a percent mean absolute error of {}%'.format(elapsed_time_sum,pmae))

Even though the cluster selection looks unnatural, the local models produce lower error and require less overall fitting time compared to the global model. But now let's revisit the cluster selection. The reason that the cluster selection appeared unnatural is that PC1 covers a wider range in the input space compared to PC2. This is because PCA operates designates dimensions in order of decreasing variance. Therefore, in order to produce a more "natural" cluster selection, we need to mitigate the dominance of PC1 in the input space. This can be accomplished through normalization.

In [ ]:
X_train_local={}
X_test_local={}
y_train_local={}
y_test_local={}
preds={}
err={}
elapsed_time=np.zeros(n_clusters)

X_total=np.append(X_train,X_test,axis=0)
max_X=X_total.max(axis=0)
min_X=X_total.min(axis=0)
X_train=(X_train-min_X)/(max_X-min_X)
X_test=(X_test-min_X)/(max_X-min_X)

cluster_obj=KMeans(n_clusters=n_clusters).fit(X_train)
clusters_train=cluster_obj.predict(X_train)
clusters_test=cluster_obj.predict(X_test)

X_train=X_train*(max_X-min_X)+min_X
X_test=X_test*(max_X-min_X)+min_X

for i in range(n_clusters):
    X_train_local[i]=X_train[np.where(clusters_train==i)[0]]
    X_test_local[i]=X_test[np.where(clusters_test==i)[0]]
    y_train_local[i]=y_train[np.where(clusters_train==i)[0]]
    y_test_local[i]=y_test[np.where(clusters_test==i)[0]]
    
plt.figure()
ax = plt.subplot(111)
for i in range(n_clusters):
    ax.plot(X_train_local[i][:,0],X_train_local[i][:,1],'.')
    
plt.xlabel('PC1')
plt.ylabel('PC2')
ax.grid(False)
plt.title('Training set inputs in the PC1-PC2 space')

plt.figure()
ax = plt.subplot(111)
for i in range(n_clusters):
    ax.plot(X_test_local[i][:,0],X_test_local[i][:,1],'.')
    
plt.xlabel('PC1')
plt.ylabel('PC2')
ax.grid(False)
plt.title('Testing set inputs in the PC1-PC2 space')

That looks better, right? Now let's see if clustering the data in a way that looks natural actually produces better models.

In [ ]:
for i in range(n_clusters):
    gp=GaussianProcessRegressor(kernel=kernel)
    time_start=time.clock()
    gp=gp.fit(X_train_local[i],y_train_local[i])
    time_stop=time.clock()
    elapsed_time[i]=time_stop-time_start
    preds[i]=gp.predict(X_test_local[i])
    err[i]=abs(y_test_local[i]-preds[i])
    
for i in range(n_clusters):
    if i==0:
        error_total=err[i]
        y_test_total=y_test_local[i]
        preds_total=preds[i]
    else:
        error_total=np.append(error_total,err[i])
        y_test_total=np.append(y_test_total,y_test_local[i])
        preds_total=np.append(preds_total,preds[i])

elapsed_time_sum=np.round(np.sum(elapsed_time),2)
pmae=np.round(100*np.sum(error_total)/np.sum(y_test_total),3)

plt.figure()
ax = plt.subplot(111)
ax.plot(y_test_total,y_test_total,'k')
ax.plot(y_test_total,preds_total,'b.')
plt.xlabel('FE simulation value')
plt.ylabel('GPR model prediction')
ax.grid(False)
plt.title('Macroscopic stress of the test set microstructures (200)')

print('The model took {} seconds to fit and produced a percent mean absolute error of {}%'.format(elapsed_time_sum,pmae))

The results are even better than before, which shows that it's important to pay attention to how your input data is clustered when constructing local models. The final local models approach we will examine is generating a model for each testing input. In this method, a number of neighbors (k) is specified. For each testing input, a model is fit to its k nearest neighbors from the training set in input space.

In [ ]:
from sklearn.neighbors import BallTree

X_train=(X_train-min_X)/(max_X-min_X)
X_test=(X_test-min_X)/(max_X-min_X)

n_neighbors=50

my_tree=BallTree(X_train)
dist,ind=my_tree.query(X_test,k=n_neighbors)

X_train=X_train*(max_X-min_X)+min_X
X_test=X_test*(max_X-min_X)+min_X

Let's take a look at how the models will be constructed for different testing points.

In [ ]:
plot_index=0

plt.figure()
ax = plt.subplot(111)
ax.plot(X_train[:,0],X_train[:,1],'k.')
ax.plot(X_train[ind[plot_index],0],X_train[ind[plot_index],1],'b.')
ax.plot(X_test[plot_index,0],X_test[plot_index,1],'ro')
plt.xlabel('PC1')
plt.ylabel('PC2')
ax.legend(['Unused training points','Used training points','Testing point'],bbox_to_anchor=(1.1, 0.65))
ax.grid(False)
plt.title('Model constructruction for index {} in the testing set'.format(plot_index))

Now let's see how this approach compares to the previous approaches we've covered.

In [ ]:
preds=np.zeros(X_test.shape[0])
elapsed_time=np.zeros(X_test.shape[0])

for i in range(X_test.shape[0]):
    gp=GaussianProcessRegressor(kernel=kernel)
    time_start=time.clock()
    gp=gp.fit(X_train[ind[i]],y_train[ind[i]])
    time_stop=time.clock()
    elapsed_time[i]=time_stop-time_start
    preds[i]=gp.predict(X_test[i].reshape(1,-1))
    
pmae=np.round(100*np.sum(abs(y_test-preds))/np.sum(y_test),3)
elapsed_time_sum=np.round(np.sum(elapsed_time),2)

plt.figure()
ax = plt.subplot(111)
ax.plot(y_test,y_test,'k')
ax.plot(y_test,preds,'b.')
plt.xlabel('FE simulation value')
plt.ylabel('GPR model prediction')
ax.grid(False)
plt.title('Macroscopic stress of the test set microstructures (200)')

print('The model took {} seconds to fit and produced a percent mean absolute error of {}%'.format(elapsed_time_sum,pmae))

As we can see, in this case the model takes significantly longer to fit (because of the 200 individual models) without offering an accuracy improvement over the three clusters approach. Therefore, this approach is not optimal for this dataset; however, there may be instances in which this approach works well.